In [1]:
import requests
import bs4
import pandas as pd
import os

In [2]:
import datetime

now = datetime.datetime.now()


Today_time = now.strftime("%H:%M")

Today_date = now.strftime("%Y-%m-%d")

In [3]:
def record_date_time(z):
    x=str.split(z,',')
    date =x[0]
    time= x[1] +':00'

    return date,time

def record_URL(x):
    y='https://timesofindia.indiatimes.com'
    url = y+x 
    return url

In [4]:
link = 'https://timesofindia.indiatimes.com/business'
res = requests.get(link)
soup = bs4.BeautifulSoup(res.text,'lxml')

In [5]:
List = soup.findAll('h2',{'class':'heading1'})

In [7]:
records = []
for link in List[2:-2]:
    try:
        category =link.text.lower()[:-5]
        url = record_URL(link.find('a')['href'])
        res = requests.get(url)
        soup =bs4.BeautifulSoup(res.text,'lxml')
        main_content = soup.select('.main-content')[0].findAll('li')

        for record in main_content[:30]:
            try:
                media_content = record.find('span',{'class':'w_tle'})

                Heading = media_content.text
                URL= record_URL(media_content.find('a')['href'])
                if(Heading[:1]=='\n'):
                    Heading = Heading[1:]

                Date = Today_date
                Time = Today_time
                Category = category
                Source = 'Times of India'
            except:
                continue
            records.append((Source,Heading,Category,Date,Time,URL))
    except:
        continue

In [8]:
import pandas as pd
df=pd.DataFrame(records,columns=['Source','Heading','Category','Date','Time','URL'])
df['Date'] = pd.to_datetime(df['Date'])

In [9]:
outname ='TOI_business-'+str(Today_date)+'.csv'

root = 'CSV/'
if not os.path.exists(root):
    os.mkdir(root)
date_today= Today_date +'/'
outdir=root+ date_today[:-1]
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)

In [10]:
df.to_csv(fullname,index=False,encoding='utf-8')
